In [1]:
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import os

In [12]:
load_urls = ['2014_nba.csv', '2015_nba.csv', '2016_nba.csv']
column_names = ['column_a', 'player', 'year', 'draft_pick', 'height_no_shoes', 'height_with_shoes', 'wingspan', 'standing_reach',\
                'vertical_max', 'vertical_max_reach', 'vertical_no_step', 'vertical_no_step_reach', 'weight', 'body_fat',\
                'hand_length', 'hand_width', 'bench', 'agility', 'sprint']



SyntaxError: invalid syntax (<ipython-input-12-d4f947cb4037>, line 4)

In [3]:
# Read single year crime files and store into final data frame
nba14_csv = "Resources/2014_nba.csv"
nba15_csv = "Resources/2015_nba.csv"
nba16_csv = "Resources/2016_nba.csv"
    
nba14_df = pd.read_csv(nba14_csv)
nba15_df = pd.read_csv(nba15_csv)
nba16_df = pd.read_csv(nba16_csv)


    


In [4]:
frames = [nba14_df, nba15_df, nba16_df]
combine_df = pd.concat(frames)
combine_df

,Unnamed: 0,Player,Year,Draft pick,Height (No Shoes),Height (With Shoes),Wingspan,Standing reach,Vertical (Max),Vertical (Max Reach),Vertical (No Step),Vertical (No Step Reach),Weight,Body Fat,Hand (Length),Hand (Width),Bench,Agility,Sprint
0,0,Aaron Craft,2014,NaN,72.75,73.75,74.00,94.5,32.0,126.5,29.0,123.5,192.0,6.1,7.75,8.50,NaN,10.78,3.26
1,1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,NaN,10.81,3.27
2,2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN,NaN
3,3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,NaN,12.05,3.48
4,4,Alex Kirk,2014,NaN,81.25,82.75,87.50,109.5,31.5,141.0,28.5,138.0,252.0,13.6,9.25,10.50,NaN,11.20,3.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,56,Troy Williams,2016,NaN,77.75,78.75,80.25,102.0,36.0,138.0,30.0,132.0,218.0,10.0,8.25,9.25,NaN,10.72,3.35
57,57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,NaN,10.80,3.20
58,58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,NaN,10.45,3.19
59,59,Wayne Selden,2016,NaN,76.25,77.75,82.50,101.0,NaN,NaN,NaN,NaN,232.0,7.3,9.00,9.25,NaN,NaN,NaN


In [11]:
combine_df = combine_df[combine_df['Draft pick'].notna()]
combine_df

,Unnamed: 0,Player,Year,Draft pick,Height (No Shoes),Height (With Shoes),Wingspan,Standing reach,Vertical (Max),Vertical (Max Reach),Vertical (No Step),Vertical (No Step Reach),Weight,Body Fat,Hand (Length),Hand (Width),Agility,Sprint
1,1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,10.81,3.27
2,2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN
3,3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,12.05,3.48
6,6,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,11.30,3.22
7,7,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,11.39,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,54,Taurean Prince,2016,12.0,78.50,79.75,83.50,102.0,36.0,138.0,34.0,136.0,220.0,6.5,9.00,8.50,10.96,3.24
55,55,Thon Maker,2016,10.0,83.75,84.75,87.00,110.5,36.5,147.0,32.0,142.5,216.0,6.1,8.75,9.50,11.15,3.33
57,57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,10.80,3.20
58,58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,10.45,3.19


In [28]:
 rds_connection_string = "postgres:dYe$4pball@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [29]:
engine.table_names() 

[]

In [30]:
combine_df.rename(columns={'Height (No Shoes)': 'Height No Shoes', 'Height (With Shoes)': 'Height With Shoes', 'Vertical (Max)': 'Vertical Max', 'Vertical (Max Reach)': 'Vertical Max Reach', 'Vertical (No Step)': 'Vertical No Step', 'Vertical (No Step Reach)': 'Vertical No Step Reach', 'Hand (Length)': 'Hand Length', 'Hand (Width)': 'Hand Width'}, inplace=True)

In [31]:
combine_df

,Player,Year,Draft pick,Height No Shoes,Height With Shoes,Wingspan,Standing reach,Vertical Max,Vertical Max Reach,Vertical No Step,Vertical No Step Reach,Weight,Body Fat,Hand Length,Hand Width,Agility,Sprint
1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,10.81,3.27
2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN
3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,12.05,3.48
6,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,11.30,3.22
7,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,11.39,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,Taurean Prince,2016,12.0,78.50,79.75,83.50,102.0,36.0,138.0,34.0,136.0,220.0,6.5,9.00,8.50,10.96,3.24
55,Thon Maker,2016,10.0,83.75,84.75,87.00,110.5,36.5,147.0,32.0,142.5,216.0,6.1,8.75,9.50,11.15,3.33
57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,10.80,3.20
58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,10.45,3.19


In [32]:
combine_df.to_sql(name='combine_df', con=engine, if_exists='append', index=False)

In [33]:
engine.table_names() 

['combine_df']

In [34]:
pd.read_sql_query('select * from combine_df', con=engine).head()

,Player,Year,Draft pick,Height No Shoes,Height With Shoes,Wingspan,Standing reach,Vertical Max,Vertical Max Reach,Vertical No Step,Vertical No Step Reach,Weight,Body Fat,Hand Length,Hand Width,Agility,Sprint
0,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,10.81,3.27
1,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN
2,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,12.05,3.48
3,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,11.30,3.22
4,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,11.39,3.37
